<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/LLM_GGUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- https://github.com/ggerganov/ggml/blob/master/docs/gguf.md gguf规范文档
- https://github.com/weedge/ggml/blob/master/docs/gguf-cn.md gguf规范中文版
- [用于量化 GGUF 模型的量化格式](https://github.com/ggerganov/ggml/blob/master/src/ggml-quants.h)。


# GGUF 工具


In [1]:
!git clone https://github.com/antirez/gguf-tools.git

Cloning into 'gguf-tools'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 136 (delta 81), reused 102 (delta 49), pack-reused 0
Receiving objects: 100% (136/136), 59.25 KiB | 4.94 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [2]:
%cd gguf-tools

/content/gguf-tools


In [3]:
!make

cc gguf-tools.c gguflib.c sds.c fp16.c \
	-march=native -flto -ffast-math \
	-g -ggdb -Wall -W -pedantic -O3 -o gguf-tools
gguf-tools.c: In function ‘gguf_tools_show’:
gguf-tools.c:169:36: warning: format ‘%llu’ expects argument of type ‘long long unsigned int’, but argument 5 has type ‘uint64_t’ {aka ‘long unsigned int’} []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wformat=]8;;]
  169 |         printf("%s tensor %.*s @%llu, %llu weights, %llu bytes\n",
      |                                 ~~~^
      |                                    |
      |                                    long long unsigned int
      |                                 %lu
......
  173 |             tensor.offset,
      |             ~~~~~~~~~~~~~           
      |                   |
      |                   uint64_t {aka long unsigned int}
gguf-tools.c:169:42: warning: format ‘%llu’ expects argument of type ‘long long unsigned int’, but argument 6 has type ‘uint64_t’ {ak

In [4]:
!pip3 install huggingface-hub

In [5]:
!huggingface-cli download TheBloke/phi-2-GGUF phi-2.Q8_0.gguf --local-dir ./models --local-dir-use-symlinks False


Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/phi-2-GGUF/resolve/main/phi-2.Q8_0.gguf to /root/.cache/huggingface/hub/tmp8bsvbj_7
phi-2.Q8_0.gguf: 100% 2.96G/2.96G [00:11<00:00, 250MB/s]
./models/phi-2.Q8_0.gguf


## gguf-tools show file.gguf
显示有关 GGUF 文件的详细信息。这将包括所有键值对，包括数组和详细的张量信息。张量的偏移量将相对于文件的开头（因此它们实际上是绝对偏移量），而不是像 GGUF 格式中的数据部分的开头。



In [9]:
!./gguf-tools show models/phi-2.Q8_0.gguf | head -100

models/phi-2.Q8_0.gguf (ver 3): 20 key-value pairs, 325 tensors
general.architecture: [string] phi2
general.name: [string] Phi2
phi2.context_length: [uint32] 2048
phi2.embedding_length: [uint32] 2560
phi2.feed_forward_length: [uint32] 10240
phi2.block_count: [uint32] 32
phi2.attention.head_count: [uint32] 32
phi2.attention.head_count_kv: [uint32] 32
phi2.attention.layer_norm_epsilon: [float32] 0.000010
phi2.rope.dimension_count: [uint32] 32
general.file_type: [uint32] 7
tokenizer.ggml.add_bos_token: [bool] false
tokenizer.ggml.model: [string] gpt2
tokenizer.ggml.tokens: [array] [!, ", #, $, %, &, ', (, ), *, +, ,, -, ., /, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, :, ;, <, =, >, ... 51170 more items of 51200]
tokenizer.ggml.token_type: [array] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 51170 more items of 51200]
tokenizer.ggml.merges: [array] [Ġ t, Ġ a, h e, i n, r e, o n, Ġt he, e r, Ġ s, a t, Ġ w, Ġ o, e n, Ġ c, i t, i s, a n, o r, e s, Ġ b, e 

## gguf-tools compare file1.gguf file2.gguf
此工具有助于了解两个 LLM（或其他以 GGUF 文件形式分发的模型）是否相关，例如一个是否是另一个的微调，或者两者是否都是从同一个父模型进行微调的。

对于每个匹配的张量（名称和参数数量相同），该命令计算平均权重差异（以百分比表示，因此在 -N 到 +N 范围内的随机分布平均上与另一个随机分布相比较将达到 100% 差异）。这有助于查看模型是否是另一个模型的微调版本，它被微调了多少，哪些层在微调时被冻结等。请注意，由于量化，即使功能上等效的张量可能也存在一些小的平均差异。

示例输出：


In [10]:
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q8_0.gguf \
  --local-dir ./models --local-dir-use-symlinks False


Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q8_0.gguf to /root/.cache/huggingface/hub/tmpexfcef09
mistral-7b-instruct-v0.2.Q8_0.gguf: 100% 7.70G/7.70G [00:28<00:00, 273MB/s]
./models/mistral-7b-instruct-v0.2.Q8_0.gguf


In [11]:
!huggingface-cli download TheBloke/SOLAR-10.7B-Instruct-v1.0-uncensored-GGUF solar-10.7b-instruct-v1.0-uncensored.Q8_0.gguf \
  --local-dir ./models --local-dir-use-symlinks False


Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/SOLAR-10.7B-Instruct-v1.0-uncensored-GGUF/resolve/main/solar-10.7b-instruct-v1.0-uncensored.Q8_0.gguf to /root/.cache/huggingface/hub/tmp6ymjwy_8
(…)10.7b-instruct-v1.0-uncensored.Q8_0.gguf: 100% 11.4G/11.4G [03:57<00:00, 48.0MB/s]
./models/solar-10.7b-instruct-v1.0-uncensored.Q8_0.gguf


In [12]:
!./gguf-tools compare models/mistral-7b-instruct-v0.2.Q8_0.gguf models/solar-10.7b-instruct-v1.0-uncensored.Q8_0.gguf


[token_embd.weight]: avg weights difference: 44.539944%
[blk.0.attn_q.weight]: avg weights difference: 48.717736%
[blk.0.attn_k.weight]: avg weights difference: 56.201885%
[blk.0.attn_v.weight]: avg weights difference: 47.087249%
[blk.0.attn_output.weight]: avg weights difference: 47.663048%
[blk.0.ffn_gate.weight]: avg weights difference: 37.508761%
[blk.0.ffn_up.weight]: avg weights difference: 39.061584%
[blk.0.ffn_down.weight]: avg weights difference: 39.632648%
[blk.0.attn_norm.weight]: avg weights difference: 3.997229%
[blk.0.ffn_norm.weight]: avg weights difference: 5.686371%
[blk.1.attn_q.weight]: avg weights difference: 47.318189%
[blk.1.attn_k.weight]: avg weights difference: 44.031983%
[blk.1.attn_v.weight]: avg weights difference: 36.303868%
[blk.1.attn_output.weight]: avg weights difference: 40.389659%
[blk.1.ffn_gate.weight]: avg weights difference: 38.658373%
[blk.1.ffn_up.weight]: avg weights difference: 38.838259%
[blk.1.ffn_down.weight]: avg weights difference: 37.840

## gguf-tools inspect-tensor file.gguf tensor.name [count]
显示指定张量的所有权重值（如果未指定 count，则仅显示前 count 个）。这对于低级别的任务很有用，比如检查量化是否按预期工作，查看引入的误差，模型指纹等。



In [20]:
!echo -e "\n----token_embd.weight----\n"
!./gguf-tools inspect-tensor models/mistral-7b-instruct-v0.2.Q8_0.gguf token_embd.weight 50
!echo -e "\n----blk.0.attn_q.weight----\n"
!./gguf-tools inspect-tensor models/mistral-7b-instruct-v0.2.Q8_0.gguf blk.0.attn_q.weight 50


----token_embd.weight----

0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 

----blk.0.attn_q.weight----

0.000085, 0.000766, 0.000000, -0.000680, 
0.001616, -0.000085, 0.003828, 0.000255, 
0.002637, 0.000000, 0.000425, -0.000085, 
0.000000, 0.000170, 0.006039, -0.000595, 
0.000000, -0.002892, 0.000000, -0.006294, 
-0.004848, -0.002382, 0.010802, 0.002892, 
0.001191, 0.000000, -0.000255, 0.000000, 
0.000255, -0.003402, 0.000000, 0.007400, 
0.000000, 0.000000, 0.002815, 0.000088, 
0.000000, 0.000880, 0.000000, 0.000000, 
0.000000,

## gguf-tools split-mixtral 65230776370407150546470161412165 mixtral.gguf out.gguf
从 Mixtral 7B MoE 中提取一个 7B 模型 out.gguf，使用指定的每层 MoE ID（在序列 652... 中有 32 个数字）。

请注意，通过 split-mixtral 方式获得的模型实际上没有执行任何有用的工作。这只是一个实验和一个不太重要的任务，用来展示如何使用该库。很可能很快就会被移除，一旦我有更有趣和有用的示例要展示，比如模型合并。

In [21]:
!./gguf-tools split-mixtral 65230776370407150546470161412165 \
  models/mistral-7b-instruct-v0.2.Q8_0.gguf models/mistral-7b-instruct-v0.2.Q8_0_out.gguf

Copying general.architecture
Copying general.name
Copying llama.context_length
Copying llama.embedding_length
Copying llama.block_count
Copying llama.feed_forward_length
Copying llama.rope.dimension_count
Copying llama.attention.head_count
Copying llama.attention.head_count_kv
Copying llama.attention.layer_norm_rms_epsilon
Copying llama.rope.freq_base
Copying general.file_type
Copying tokenizer.ggml.model
Copying tokenizer.ggml.tokens
Copying tokenizer.ggml.scores
Copying tokenizer.ggml.token_type
Copying tokenizer.ggml.bos_token_id
Copying tokenizer.ggml.eos_token_id
Copying tokenizer.ggml.unknown_token_id
Copying tokenizer.ggml.padding_token_id
Copying tokenizer.ggml.add_bos_token
Copying tokenizer.ggml.add_eos_token
Copying tokenizer.chat_template
Copying general.quantization_version
Skipping tensor blk.0.ffn_gate.weight
Skipping tensor blk.0.ffn_up.weight
Skipping tensor blk.0.ffn_down.weight
Skipping tensor blk.1.ffn_gate.weight
Skipping tensor blk.1.ffn_up.weight
Skipping tensor 